In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
!pip install -e ../.

Obtaining file:///Users/shanto/LFL/SQuADDS/SQuADDS
  Preparing metadata (setup.py) ... done
  Attempting uninstall: SQuADDS
    Found existing installation: SQuADDS 0.1
    Uninstalling SQuADDS-0.1:
      Successfully uninstalled SQuADDS-0.1
  Running setup.py develop for SQuADDS


# Tutorial 2: Contributing to SQuADDS


**Table of Contents:**

0. [Contribution Information Setup](#setup)
1. [Understanding the terminology and database structure](#structure)
2. [Contributing to an existing database node](#contribution)
3. [Creating new database node](#creation)
4. [Building on top of others works](#missing)

---

## Contribution Information Setup <a name="setup"></a>

In order to contribute to SQuADDS, you will need to provide some information about yourself. This information will be used to track your contributions and to give you credit for your work. You can provide this information by updating the following variables in the `.env` file in the root directory of the repository:

```
GROUP_NAME = ""
PI_NAME = ""
INSTITUTION = ""
USER_NAME = ""
```

Or you can provide this information by executing the following cell.


In [17]:
from squadds.database import *

In [ ]:
create_contributor_info()

## Understanding the terminology and database structure <a name="structure"></a>

- HuggingFace
- Datasets
- Configurations
- Structure of SQuADDS_DB
- Adding to SQuADDS_DB

## Contributing to an existing database node <a name="contribution"></a>

### Hugging Face Native Solution:

**To append to an existing configuration:**

1. Modify the existing `YourDatasetConfig` if needed.
2. Update the `_split_generators` and `_generate_examples` methods to include your additions.
3. Test your changes locally to ensure the dataset loads correctly.
4. Update the documentation (`README.md`) to include your new configuration.
5. Create a pull request with your changes.

### SQuADDS API:

In [6]:
import glob

## Creating a new database node <a name="creation"></a>

### HuggingFace Native Solution:

One can use workflows developed by HuggingFace to create a new **configuration** to the SQuADDS_DB Dataset. The following is a list of steps to follow: 

**To add a new configuration:**

1. Add a new `YourDatasetConfig` to `BUILDER_CONFIGS` with a unique name, version, and description.
2. Update `_URLs` with the download URL for the new configuration.
3. Modify `_split_generators` and `_generate_examples` methods to handle the new configuration.
4. Test your changes locally to ensure the dataset loads correctly.
5. Update the documentation (`README.md`) to include your new configuration.
6. Create a pull request with your changes.


### SQuADDS API:

You can also use the SQuADDS API to create a new configuration. The following cells will walk you through the process.

#### Step 1: Standardizing your data

We want the data to be in a `json` format with **AT LEAST** to have the following fields. You can add as many more supplementary fields as you want.

```json
{
    "design":{
        "design_options": design_options,
        "design_tool": design_tool_name, 
    },
    "sim_options":{
        "setup": sim_setup_options,
        "simulator": simulator_name,
    },
    "sim_results":{
        "result1": sim_result1,
        "unit1": unit1,
        "result2": sim_result2,
        "unit2": unit2,
    },
    "contributor":{
        "group": group_name,
        "PI": pi_name,
        "institution": institution,
        "uploader": user_name,
        "date_created": "YYYY-MM-DD-HHMMSS",
    },
}
```

If all the `sim_results` has the same units you can just use a `"units":units` field instead of repeating the unit for each result.



Example Qiskit Metal Rendering Code to get `design`

Example Simulation Code to get `sim_results`

Use of SQuADDS API to upload dataset - adds `contributor` information automatically.

Generate the `load_dataset` config file

Upload to `SQuADDS_DB`

In [79]:
def combine_json_files(source_directory, output_file):
    all_data = []

    # List all JSON files in the directory
    file_paths = glob.glob(os.path.join(source_directory, "*.json"))

    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            all_data.append(data)

    # Write combined data to a single file
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(all_data, outfile, indent=4)



In [80]:
source_directory = "../data/qubit/TransmonCross/cap_matrix"  # Replace with your source directory
output_file = "../data/combined_qubit_data.json"  # Replace with your desired output file path
combine_json_files(source_directory, output_file)

In [81]:
source_directory = "../data/coupler/NCap/cap_matrix"  # Replace with your source directory
output_file = "../data/combined_coupler_data.json"  # Replace with your desired output file path
combine_json_files(source_directory, output_file)

In [82]:
source_directory = "../data/cavity_claw/RouteMeander/eigenmode/"  # Replace with your source directory
output_file = "../data/combined_cavity-claw_data.json"  # Replace with your desired output file path
combine_json_files(source_directory, output_file)

In [83]:
# rename the json files in data to "LFL_USC_{hash}.json" where the hash is based on the file contents

for file in glob.glob("../data/*.json"):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        hash = hashlib.md5(json.dumps(data, sort_keys=True).encode('utf-8')).hexdigest()
        new_file = file.replace(".json", f"LFL_USC_{hash}.json")
        os.rename(file, new_file)

In [99]:
!datasets-cli test ../data/SQuADDS_DB.py --save_info --all_configs

Loading Dataset Infos from /Users/shanto/.cache/huggingface/modules/datasets_modules/datasets/SQuADDS_DB/6c042e99be0aa47463aa5d9ae2ceeb87b02331a7e2aa159865c27f7b19e9316e
Testing builder 'qubit-TransmonCross-cap_matrix' (1/3)
Generating dataset s_qu_adds_db (/Users/shanto/.cache/huggingface/datasets/s_qu_adds_db/qubit-TransmonCross-cap_matrix/1.0.0/6c042e99be0aa47463aa5d9ae2ceeb87b02331a7e2aa159865c27f7b19e9316e)
Checksum Computation took 0.0 min
Extracting data files: 100%|█████████████████████| 3/3 [00:00<00:00, 853.43it/s]
Generating train split
Traceback (most recent call last):
  File "/Users/shanto/miniconda3/envs/qiskit_metal/bin/datasets-cli", line 8, in <module>
    sys.exit(main())
  File "/Users/shanto/miniconda3/envs/qiskit_metal/lib/python3.10/site-packages/datasets/commands/datasets_cli.py", line 39, in main
    service.run()
  File "/Users/shanto/miniconda3/envs/qiskit_metal/lib/python3.10/site-packages/datasets/commands/test.py", line 146, in run
    builder.download_and

In [96]:
import json
import random
import os

def create_train_val_test_splits(source_files, output_directory, train_ratio=0.7, val_ratio=0.15):
    """
    Splits the data from source JSON files into train, validation, and test sets.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    all_data = []

    # Load data from all source files
    for file_path in source_files:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            all_data.extend(data)

    # Shuffle the data
    random.shuffle(all_data)

    # Split the data
    total_data = len(all_data)
    train_end = int(total_data * train_ratio)
    val_end = train_end + int(total_data * val_ratio)

    train_data = all_data[:train_end]
    val_data = all_data[train_end:val_end]
    test_data = all_data[val_end:]

    # Save the splits
    with open(os.path.join(output_directory, 'train.json'), 'w', encoding='utf-8') as f:
        json.dump(train_data, f, indent=4)

    with open(os.path.join(output_directory, 'validation.json'), 'w', encoding='utf-8') as f:
        json.dump(val_data, f, indent=4)

    with open(os.path.join(output_directory, 'test.json'), 'w', encoding='utf-8') as f:
        json.dump(test_data, f, indent=4)



In [97]:
source_files = [
    "/Users/shanto/LFL/SQuADDS/SQuADDS/data/cavity_claw/LFL_USC_cavity_claw_3e95ba4a2e4da2141f9edaa9f9fa1653.json",
    "/Users/shanto/LFL/SQuADDS/SQuADDS/data/coupler/LFL_USC_coupler_e7855e5c7467f76edb09779d8f3a1a0c.json",
    "/Users/shanto/LFL/SQuADDS/SQuADDS/data/qubit/LFL_USC_qubit_e68f323df894ba4b2891bd64742a2c35.json",
]
output_directory = '../data/'

create_train_val_test_splits(source_files, output_directory)